# Importar Librarias y datos

In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np   
import itertools

from scipy.stats import chi2_contingency, ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf 

In [50]:
df = pd.read_parquet("output_04.parquet")

In [51]:
df.head()

,age,gender,education,marital_status,race,is_hispanic,employment_commitment,employment_stat,wage_per_hour,working_week_per_year,...,tax_status,gains,losses,stocks_status,citizenship,migration_year,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,income_above_limit
0,79,1,secondary_complete,Widowed,White,All other,Not in labor force,0,0,52,...,Head of household,0,0,292,Native,95,US,?,?,0
1,65,1,secondary_complete,Widowed,White,All other,Children or Armed Forces,0,0,0,...,Single,0,0,0,Native,94,US,unchanged,unchanged,0
2,21,0,secondary_incomplete,Never married,Black,All other,Children or Armed Forces,0,500,15,...,Single,0,0,0,Native,94,US,unchanged,unchanged,0
3,2,1,child,Never married,Asian or Pacific Islander,All other,Children or Armed Forces,0,0,0,...,Nonfiler,0,0,0,Native,94,US,unchanged,unchanged,0
4,70,0,secondary_complete,Married-civilian spouse present,White,All other,Not in labor force,0,0,0,...,Joint both 65+,0,0,0,Native,95,US,?,?,0


# Revisión de los datos

Los nativos no deberían tener "migration_year"

Se verificó el año y se compararon las otras columnas de migración:

- `'migration_code_change_in_msa'`
- `'migration_code_change_in_reg'`  


Se eliminó esta columna de año porque muestra que se registró la información de migración o, si no se registró información, el año se indica como `"95"` .

Parece que mig_year (1994 o 1995) está relacionado con la columna migration_code_change_in_msa. Para entenderlo mejor, desglosémoslo:

Censo de EE.UU.: Posiblemente, los datos provienen del Current Population Survey (CPS) o del Censo de EE.UU., que suelen incluir preguntas sobre migración interna.
Columna migration_code_change_in_msa:
"MSA" significa Metropolitan Statistical Area (Área Estadística Metropolitana).
Esta columna probablemente indica si una persona cambió de MSA en el último año.
Si tiene algún dato (un código), podría significar que hubo un cambio de MSA.
Si tiene "?", podría significar que no hay información disponible o que la persona no migró.

Posible conexión con mig_year:

Cambio en la forma de recopilar datos
Puede que en 1994 el censo registrara información más detallada sobre migración dentro de las MSA.
En 1995, es posible que haya habido un cambio en la encuesta, lo que resultó en valores "?" para esa columna.

Diferencias en la población encuestada:

Si mig_year = 1994 significa que la persona respondió a la encuesta en 1994, entonces en ese año las personas tenían datos en migration_code_change_in_msa.
Si mig_year = 1995, quizás la encuesta cambió y ya no se preguntó o registró de la misma manera.

In [55]:
df.columns

Index(['age', 'gender', 'education', 'marital_status', 'race', 'is_hispanic',
       'employment_commitment', 'employment_stat', 'wage_per_hour',
       'working_week_per_year', 'industry_code_main', 'occupation_code',
       'total_employed', 'household_stat', 'tax_status', 'gains', 'losses',
       'stocks_status', 'citizenship', 'migration_year',
       'country_of_birth_own', 'migration_code_change_in_msa',
       'migration_code_change_in_reg', 'income_above_limit'],
      dtype='object')

In [56]:
df[['citizenship', 'country_of_birth_own', 'migration_code_change_in_msa', 'migration_code_change_in_reg', 'migration_year']][(df['citizenship'] == "Native") & (df['migration_code_change_in_msa'] == '?')].head(25)

,citizenship,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,migration_year
0,Native,US,?,?,95
4,Native,US,?,?,95
6,Native,US,?,?,95
7,Native,US,?,?,95
8,Native,US,?,?,95
9,Native,US,?,?,95
10,Native,US,?,?,95
12,Native,US,?,?,95
16,Native,US,?,?,95
17,Native,US,?,?,95


In [57]:
df[['citizenship', 'country_of_birth_own', 'migration_code_change_in_msa', 'migration_code_change_in_reg', 'migration_year']][(df['citizenship'] == "Native") & (df['migration_code_change_in_msa'] != '?')].head(25)

,citizenship,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,migration_year
1,Native,US,unchanged,unchanged,94
2,Native,US,unchanged,unchanged,94
3,Native,US,unchanged,unchanged,94
11,Native,US,unchanged,unchanged,94
13,Native,US,unchanged,unchanged,94
14,Native,US,unchanged,unchanged,94
15,Native,US,unchanged,unchanged,94
18,Native,US,unchanged,unchanged,94
19,Native,US,unchanged,unchanged,94
22,Native,US,unchanged,unchanged,94


In [58]:
df_copia = df.drop(columns=['migration_year'])

# Filtro de edad

La edad legal para trabajar en EE. UU. es de 14 años. Filtra los resultados para incluir solo a las personas que están legalmente autorizadas a ganar un salario.

In [60]:
df_copia_mas_que_14 = df_copia[df_copia['age'] > 14]

In [61]:
df_copia_mas_que_14.head()

,age,gender,education,marital_status,race,is_hispanic,employment_commitment,employment_stat,wage_per_hour,working_week_per_year,...,household_stat,tax_status,gains,losses,stocks_status,citizenship,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,income_above_limit
0,79,1,secondary_complete,Widowed,White,All other,Not in labor force,0,0,52,...,Householder,Head of household,0,0,292,Native,US,?,?,0
1,65,1,secondary_complete,Widowed,White,All other,Children or Armed Forces,0,0,0,...,Nonfamily Householder,Single,0,0,0,Native,US,unchanged,unchanged,0
2,21,0,secondary_incomplete,Never married,Black,All other,Children or Armed Forces,0,500,15,...,Child,Single,0,0,0,Native,US,unchanged,unchanged,0
4,70,0,secondary_complete,Married-civilian spouse present,White,All other,Not in labor force,0,0,0,...,Spouse,Joint both 65+,0,0,0,Native,US,?,?,0
5,45,0,university_graduate_or_higher,Married-civilian spouse present,Asian or Pacific Islander,All other,Children or Armed Forces,0,825,52,...,Householder,Joint both under 65,0,0,0,Foreign born- Not a citizen of U S,Other,unchanged,unchanged,0


# Guardar output

In [63]:
df_copia_mas_que_14.to_parquet("output_06.parquet")